In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model="gpt-4.1-nano")
chat = ChatOpenAI()

a = llm.predict("how many planets are there?")
b = chat.predict("how many planets are there?")

print(a)
print("-------------")
print(b)


## Predict Messages

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-4.1-nano",
                 temperature=0.1)

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = "you are a geograpy expert. and you only reply in Korean"
    ),
    AIMessage (
        content = "안녕, 내 이름은 지피티야!"
    ),
    HumanMessage(
        content = "what is the distance between Mexico and Korea. Also, what is your name?"
    )
]

chat.predict_messages(messages)


## Prompt Templates

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(model="gpt-4.1-nano",
                 temperature=0.1)


template = ChatPromptTemplate.from_messages([
    ("system","you are a geograpy expert. and you only reply in {language}"),
    ("ai","안녕, 내 이름은 {name}!"),
    ("human","what is the distance between {country_a} and {country_b}. Also, what is your name?")
])


prompt = template.format_messages(
    language="korean",
    name="지피티",
    country_a="Mexico",
    country_b="Korea"
)

chat.predict_messages(prompt)

## OutputParser and LCEL

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(model="gpt-4.1-nano",
                 temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

    
template = ChatPromptTemplate.from_messages([
    ("system","you are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do NOT reply with anything else. "),
    ("human","{question}")
])


prompt = template.format_messages(
    max_items = 10,
    question = "what are the country?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

### Chain을 사용해서 코드를 줄여보자!

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"what are the pokemons? in Korean"
})

## Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
chat = ChatOpenAI(model="gpt-4.1-nano",
                 temperature=0.1,
                 streaming = True,
                 callbacks=[StreamingStdOutCallbackHandler()])

 
chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it and answer in Korean."),
    ("human","{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine":"US"
})